In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# For data preprocess
import numpy as np
import csv
import os

# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from utils.mylib import *
from d2l import torch as d2l


config = {
    'sava_path': 'models/MF_1M.pth',
    'batch_size': 500
}
init_Seed()
device = get_device()                 # get the current available device ('cpu' or 'cuda')

data_train = '../data/ML1M/ML1M_copy1_train.txt'
data_test = '../data/ML1M/ML1M_copy1_test.txt'

tr_set = prep_dataloader(data_train, 'train', config['batch_size'])
dv_set = prep_dataloader(data_test, 'dev', config['batch_size'])
# tt_set = prep_dataloader("data/ML100K/ML100K_copy1_test.txt", 'test', config['batch_size'], target_only=target_only)



GPU ready!
Max user: 6040
Max item: 3952
Finished reading the train set of MoviesLen Dataset (600126 samples found, each dim = 2)
Max user: 6040
Max item: 3952
Finished reading the dev set of MoviesLen Dataset (200041 samples found, each dim = 2)


In [2]:
model = MF().to(device)
ckpt = torch.load(config['sava_path'], map_location='cpu')  # Load your best model
model.load_state_dict(ckpt)
# plot_pred(dv_set, model, device)  # Show prediction on the validation set
print(dev(dv_set, model, device))

0.7528189614542324


In [7]:
torch.save(model.V, "./VTensor.pt")

y = torch.load("./VTensor.pt")